# Computing Assignment 1: Jacob Holmes

Installed the following packages to the venv before running all the code.

In [1]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

## Problem 1

### Data

In [2]:
Shift8Hours = [1,2,3,4,5,6,7] # Possible 8-hour shifts indicies
Shift6Hours = [1,2,3,4,5,6,7,8] # Possible 6-hour shifts indicies
Shift4Hours = [1,2,3,4,5,6,7,8,9] # Possible 4-hour shifts indicies
TimePeriods = [1,2,3,4,5,6,7,8,9,10] # Number of consecutive 2-hour time periods
ShiftTypes = [1,2,3] # 1=8-hour, 2=6-hour, 3=4-hour

n = 10  # Number of time periods

# Cost in $ per employee working shift type "ShiftType"
c = {
    1: 180,  # Cost for 8-hour shift
    2: 140,   # Cost for 6-hour shift
    3: 100    # Cost for 4-hour shift
}

# Number of employees required in time period "l"
r = {
    1: 2,
    2: 4,
    3: 5,
    4: 8,
    5: 11,
    6: 10,
    7: 9,
    8: 7,
    9: 6,
    10: 5
}


#Used Copilot to generate the following repetitive coverage matrices
# Coverage matrix for 8-hour shifts on time period "t"
# Each 8-hour shift covers 4 consecutive periods
a = {}
for s in Shift8Hours:
    for t in TimePeriods:
        # Shift s starts at period s, covers periods s to s+3
        a[(t, s)] = 1 if t >= s and t < s + 4 else 0

# Display the matrix for verification

#for t in TimePeriods:
#    row = [a[(t, s)] for s in Shift8Hours]
#    print(f"Time period {t}: {row}")


# Coverage matrix for 6-hour shifts on time period "t"
# Each 6-hour shift covers 3 consecutive periods
b = {}
for s in Shift6Hours:
    for t in TimePeriods:
        # Shift s starts at period s, covers periods s to s+2
        b[(t, s)] = 1 if t >= s and t < s + 3 else 0

# Display the matrix for verification
#for t in TimePeriods:
#    row = [b[(t, s)] for s in Shift6Hours]
#    print(f"Time period {t}: {row}")

# Coverage matrix for 4-hour shifts on time period "t"
# Each 4-hour shift covers 2 consecutive periods
d = {}
for s in Shift4Hours:
    for t in TimePeriods:
        # Shift s starts at period s, covers periods s to s+1
        d[(t, s)] = 1 if t >= s and t < s + 2 else 0

# Display the matrix for verification
#for t in TimePeriods:
#    row = [d[(t, s)] for s in Shift4Hours]
#    print(f"Time period {t}: {row}")

### Build the model for staffing coverage

In [3]:
m1 = pyo.ConcreteModel()

# Define index sets
m1.T = pyo.Set(initialize=TimePeriods)  # Time periods
m1.S8 = pyo.Set(initialize=Shift8Hours)  # 8-hour shifts
m1.S6 = pyo.Set(initialize=Shift6Hours)  # 6-hour shifts
m1.S4 = pyo.Set(initialize=Shift4Hours)  # 4-hour shifts
m1.ST = pyo.Set(initialize=ShiftTypes)  # Shift types

#m1. ST.pprint()

# Define parameters
m1.c = pyo.Param(m1.ST, initialize=c)  # Cost per shift type
m1.r = pyo.Param(m1.T, initialize=r)  # Required employees per time period
m1.a = pyo.Param(m1.T, m1.S8, initialize=a)  # Coverage matrix for 8-hour shifts
m1.b = pyo.Param(m1.T, m1.S6, initialize=b)  # Coverage matrix for 6-hour shifts
m1.d = pyo.Param(m1.T, m1.S4, initialize=d) # Coverage matrix for 4-hour shifts

#m1.d.pprint()

# Define decision variables
m1.x8 = pyo.Var(m1.S8, domain=pyo.NonNegativeReals)  # Number of employees on 8-hour shifts
m1.x6 = pyo.Var(m1.S6, domain=pyo.NonNegativeReals)  # Number of employees on 6-hour shifts
m1.x4 = pyo.Var(m1.S4, domain=pyo.NonNegativeReals)  # Number of employees on 4-hour shifts

# Define objective function: Minimize total cost
def obj_rule(m):
    return sum(m.c[1] * m.x8[s] for s in m.S8) + \
           sum(m.c[2] * m.x6[s] for s in m.S6) + \
           sum(m.c[3] * m.x4[s] for s in m.S4)

m1.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Define constraints

# Staffing minimum requirement constraints
def staffing_requirement(m, t):
    return sum(m.a[t, s] * m.x8[s] for s in m.S8) + \
           sum(m.b[t, s] * m.x6[s] for s in m.S6) + \
           sum(m.d[t, s] * m.x4[s] for s in m.S4) >= m.r[t]

m1.StaffingRequirement = pyo.Constraint(m1.T, rule=staffing_requirement)

# Staffing requirement for proportinality so that at least 40% of employees are on 8-hour shifts for every time period
def prop_8_hour_shift(m, t):
    return sum(m.a[t, s] * m.x8[s] for s in m.S8) >= \
        0.4 *   (sum(m.a[t, s] * m.x8[s] for s in m.S8) + \
                sum(m.b[t, s] * m.x6[s] for s in m.S6) + \
                sum(m.d[t, s] * m.x4[s] for s in m.S4))

m1.Prop8HourShift = pyo.Constraint(m1.T, rule=prop_8_hour_shift)



### Solve the model

In [4]:
opt = pyo.SolverFactory('cplex')

results1 = opt.solve(m1)

# Print the results

print("Objective value: $", pyo.value(m1.OBJ))

print("Optimal number of employees for 8-hour shift s:", {s: pyo.value(m1.x8[s]) for s in m1.S8})

print("Optimal number of employees for 6-hour shift s:", {s: pyo.value(m1.x6[s]) for s in m1.S6})

print("Optimal number of employees for 4-hour shift s:", {s: pyo.value(m1.x4[s]) for s in m1.S4})

print("The number of total employees on each time period is as follows:", {t: sum(m1.a[t, s] * pyo.value(m1.x8[s]) for s in m1.S8) + 
                            sum(m1.b[t, s] * pyo.value(m1.x6[s]) for s in m1.S6) + 
                            sum(m1.d[t, s] * pyo.value(m1.x4[s]) for s in m1.S4) for t in m1.T})


#print("The proportion of employees on 8-hour shifts for each time period is as follows:", {t: sum(m1.a[t, s] * pyo.value(m1.x8[s]) for s in m1.S8) / 
#                            (sum(m1.a[t, s] * pyo.value(m1.x8[s]) for s in m1.S8) + 
#                            sum(m1.b[t, s] * pyo.value(m1.x6[s]) for s in m1.S6) + 
#                            sum(m1.d[t, s] * pyo.value(m1.x4[s]) for s in m1.S4)) for t in m1.T})

Objective value: $ 3060.0
Optimal number of employees for 8-hour shift s: {1: 1.0, 2: 2.0, 3: 2.0, 4: 2.0, 5: 4.0, 6: 1.0, 7: 2.0}
Optimal number of employees for 6-hour shift s: {1: 0.0, 2: 0.0, 3: 0.0, 4: 1.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0}
Optimal number of employees for 4-hour shift s: {1: 1.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 3.0}
The number of total employees on each time period is as follows: {1: 2.0, 2: 4.0, 3: 5.0, 4: 8.0, 5: 11.0, 6: 10.0, 7: 9.0, 8: 7.0, 9: 6.0, 10: 5.0}


## Problem 3

### Data

In [5]:
# Indicies and parameters for indicies
n = 5 # Number of months to consider

i = [0,1,2,3,4,5] # Index for months
iy = [-2,-1,0,1,2,3,4,5] # Index for months for use with y dv
ic = [1,2,3,4,5] # Index i for use with constraints

j = [1,2,3] # Index for investment options (1-month, 2-month, 3-month)

# Parameters
r = {1: 500, 2: 800, 3: 300, 4: 300, 5: 0} # Revenue in $ received at the beginning of month i

b = {1: 600, 2: 600, 3: 450, 4: 250, 5: 0} # Bill amount in $ to pay right after recieving revenue in month i

t = {1: 1+0.002,
     2: (1+0.007)**2,
     3: (1+0.02)**3} # Growth factor for investment option j

f = 500 # Initial funds available at the beginning of month 1

### Build the model

In [6]:
m3 = pyo.ConcreteModel()

# Define index sets
m3.I = pyo.Set(initialize=i)  # Months
m3.IY = pyo.Set(initialize=iy)  # Months for y dv
m3.J = pyo.Set(initialize=j)  # Investment options
m3.IC = pyo.Set(initialize=ic)  # Index i for use with constraints

# Define parameters
m3.r = pyo.Param(m3.IC, initialize=r)  # Revenue in $ at beginning of month i
m3.b = pyo.Param(m3.IC, initialize=b)  # Bill amount in $ to pay right after recieving revenue in month i
m3.t = pyo.Param(m3.J, initialize=t)  # Growth factor for investment option j

# Define decision variables
m3.x = pyo.Var(m3.I, domain=pyo.NonNegativeReals)  # Cash on hand at beginning of month i after all transactions
m3.y = pyo.Var(m3.IY, m3.J, domain=pyo.NonNegativeReals)  # Amount invested in j-month-long option at beginning of month i

# Define objective function: Maximize cash on hand at beginning of month n
def obj_rule(m):
    return m.x[n]

m3.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.maximize)


# Define constraints

# Balancing constraints for cash on hand and cash flows
def cash_flow_balance(m, i):
    if i == 0:
        return m.x[i] == f  # Initial cash on hand is f
    else:
        return m.x[i] == m.x[i-1] + m.r[i] - m.b[i] + sum(m.y[i - j, j] * m.t[j] - m.y[i, j] for j in m.J)
    
m3.CashFlowBalance = pyo.Constraint(m3.I, rule=cash_flow_balance)

#m3.CashFlowBalance.pprint()
    
# Investment constraints: Cannot invest in months that do not exist or cannot for final months
def investment_constraint(m, i, j):
    if i <= 0:
        return m.y[i, j] == 0
    elif i == n:
        return m.y[i, j] == 0
    elif i == n-1 and j >= 2:
        return m.y[i, j] == 0
    elif i == n-2 and j == 3:
        return m.y[i, j] == 0 
    else:
        return pyo.Constraint.Skip
    
m3.InvestmentConstraint = pyo.Constraint(m3.IY, m3.J, rule=investment_constraint)

# Checking the investment opportunity constraint logic to ensure that only the months that should be restricted are restricted
#m3.InvestmentConstraint.pprint()
    




### Solve the model

In [7]:
results3 = opt.solve(m3)

# Print the results

print("Objective value: $", pyo.value(m3.OBJ))
print("Optimal cash on hand at beginning of each month i:", {i: pyo.value(m3.x[i]) for i in m3.I})
print("Optimal investment amounts y[i,j]:", {(i,j): pyo.value(m3.y[i,j]) for i in m3.IY for j in m3.J if pyo.value(m3.y[i,j]) > 0})
print("All other combinations of y[i,j] are 0")



Objective value: $ 530.3839683724885
Optimal cash on hand at beginning of each month i: {0: 500.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 530.3839683724885}
Optimal investment amounts y[i,j]: {(1, 2): 147.9218459857463, (1, 3): 252.0781540142537, (2, 3): 200.0, (4, 1): 317.5073536651582}
All other combinations of y[i,j] are 0


## Problem  5

### Data

In [8]:
i = [1,2] # Scrap yard indicies
j = [1,2] # Recycling facility indicies
k = [1,2] # Customer indicies

# Parameters

d = {1: 300, 2: 700} # Demand in tons of recycled steel for customer k

s = {(1,1): 45,
     (1,2): 70,
     (2,1): 30,
     (2,2): 35} # Cost in $ per ton of recycled steel transported from recycling facility j to customer k

p = {1: 55, 2: 40} # Price in $ per ton of scrap steel processed at recycling facility j

r = 0.95 # Number of tons of recycled steel produced per ton of scrap steel processed

t = {(1,1): 10,
     (1,2): 60,
     (2,1): 80,
     (2,2): 70} # Cost in $ per ton of scrap steel transported from scrap yard i to recycling facility j

a = {1: 500, 2: 800} # Amount of scrap steel in tons available at scrap yard i

l = 800 # Maximum processing capacity in tons at each recycling facility

### Build the model

In [9]:
# Define model
m5 = pyo.ConcreteModel()

# Define Index sets
m5.I = pyo.Set(initialize=i)  # Scrap yards
m5.J = pyo.Set(initialize=j)  # Recycling facilities
m5.K = pyo.Set(initialize=k)  # Customers

# Define Parameters
m5.d = pyo.Param(m5.K, initialize=d)  # Demand in tons for customer k
m5.s = pyo.Param(m5.J, m5.K, initialize=s)  # Cost in $ per ton from facility j to customer k
m5.p = pyo.Param(m5.J, initialize=p)  # Price in $ per ton processed at facility j
m5.r = pyo.Param(initialize=r)  # Tons of recycled steel produced per ton of scrap steel processed
m5.t = pyo.Param(m5.I, m5.J, initialize=t)  # Cost in $ per ton from scrap yard i to facility j
m5.a = pyo.Param(m5.I, initialize=a)  # Amount of scrap steel available at scrap yard i
m5.l = pyo.Param(initialize=l)  # Max processing capacity at each facility

# Define Decision Variables
m5.x = pyo.Var(m5.I, m5.J, domain=pyo.NonNegativeReals)  # Tons of scrap steel transported from scrap yard i to facility j
m5.y = pyo.Var(m5.J, m5.K, domain=pyo.NonNegativeReals)  # Tons of recycled steel transported from facility j to customer k

# Define Objective Function
def obj_rule(m):
    return sum((m.p[j] + m.t[i, j]) * m.x[i, j] for i in m.I for j in m.J) + \
           sum(m.s[j, k] * m.y[j, k] for j in m.J for k in m.K)

m5.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

#m5.OBJ.pprint()

# Define Constraints

# Customer demand constraints from recycled steel
def customer_demand(m, k):
    return sum(m.y[j, k] for j in m.J) == m.d[k]

m5.CustomerDemand = pyo.Constraint(m5.K, rule=customer_demand)

# Scrap yard supply constraints
def scrap_yard_supply(m, i):
    return sum(m.x[i, j] for j in m.J) <= m.a[i]

m5.ScrapYardSupply = pyo.Constraint(m5.I, rule=scrap_yard_supply)

# Recycling facility capacity constraints
def facility_capacity(m, j):
    return sum(m.x[i, j] for i in m.I) <= m.l

m5.FacilityCapacity = pyo.Constraint(m5.J, rule=facility_capacity)

# Recycling facility flow balance constraints
def facility_flow_balance(m, j):
    return sum(m.y[j, k] for k in m.K) == m.r * sum(m.x[i, j] for i in m.I)

m5.FacilityFlowBalance = pyo.Constraint(m5.J, rule=facility_flow_balance)


### Solve the model

In [10]:
results5 = opt.solve(m5)

# Print the results

print("Objective value: $", pyo.value(m5.OBJ))
print("Optimal tons of scrap steel transported from scrap yard i to facility j:", \
      {(i,j): pyo.value(m5.x[i,j]) for i in m5.I for j in m5.J if pyo.value(m5.x[i,j]) > 0})
print("Optimal tons of recycled steel transported from facility j to customer k:", \
      {(j,k): pyo.value(m5.y[j,k]) for j in m5.J for k in m5.K if pyo.value(m5.y[j,k]) > 0})
print("All other combinations of x[i,j] and y[j,k] are 0")

Objective value: $ 137414.47368421053
Optimal tons of scrap steel transported from scrap yard i to facility j: {(1, 1): 500.0, (2, 2): 552.6315789473684}
Optimal tons of recycled steel transported from facility j to customer k: {(1, 1): 300.0, (1, 2): 175.0, (2, 2): 525.0}
All other combinations of x[i,j] and y[j,k] are 0


## Problem 6

### Data

In [11]:
i = [1,2] # month indicies
j = [1,2] # product indicies
k = [1,2] # machine line indicies
iy = [0,1,2] # month indicies for use with y dv definition

# Parameters
c = {(1,1): 2.1, (1,2): 2.4,
     (2,1): 2.18, (2,2): 2.52} # Cost in $ per unit of product j produced on machine line k

h = 0.20 # Holding cost in $ per unit of product held in inventory from month i to month i+1

d = {(1,1): 5000, (1,2): 2000,
     (2,1): 8000, (2,2): 4000} # Demand in units of product j in month i

r = {(1,1): 0.15, (1,2): 0.17,
     (2,1): 0.12, (2,2): 0.14} # Amount of time in hours using machine line k to produce one unit of product j

l ={(1,1): 800, (1,2): 2000,
    (2,1): 500, (2,2): 1100} # Available machine line k hours in month i

b = 1000 # Base inventory required by management for each product at the end of April (month 2)

f = {1: 500, 2: 750} # Initial inventory in units of product j at the beginning of March (month 1)

### Build the model

In [12]:
# Define model
m6 = pyo.ConcreteModel()

# Define index sets
m6.I = pyo.Set(initialize=i)  # Months (1=March, 2=April)
m6.J = pyo.Set(initialize=j)  # Products (1=Product A, 2=Product B)
m6.K = pyo.Set(initialize=k)  # Machine lines
m6.IY = pyo.Set(initialize=iy)  # Months for y dv definition

# Define parameters
m6.c = pyo.Param(m6.I, m6.J, initialize=c)  # Cost in $ per unit of product j on machine line k
m6.h = pyo.Param(initialize=h)  # Holding cost in $ per unit held in inventory
m6.d = pyo.Param(m6.I, m6.J, initialize=d)  # Demand in units of product j in month i
m6.r = pyo.Param(m6.J, m6.K, initialize=r)  # Time in hours using machine line k to produce one unit of product j
m6.l = pyo.Param(m6.I, m6.K, initialize=l)  # Available machine line k hours in month i
m6.b = pyo.Param(initialize=b)  # Base inventory required at end of month 2
m6.f = pyo.Param(m6.J, initialize=f)  # Initial inventory in units of product j at beginning of month 1

# Define decision variables
m6.x = pyo.Var(m6.I, m6.J, m6.K, domain=pyo.NonNegativeReals)  # Units of product j produced in month i on line k
m6.y = pyo.Var(m6.IY, m6.J, domain=pyo.NonNegativeReals)  # Inventory of product j at end of month i

# Define objective function
def obj_rule(m):
    return sum(m.c[j, k] * m.x[i, j, k] for i in m.I for j in m.J for k in m.K) + \
           sum(m.h * m.y[i, j] for i in m.I for j in m.J)

m6.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Define constraints

# Demand fulfillment and inventory/production balance constraints
def demand_inventory_balance(m, i, j):
    if i == 0:
        return m.y[0,j] == m.f[j]
    else:
        return m.y[i,j] == m.y[i-1,j] + sum(m.x[i,j,k] for k in m.K) - m.d[i,j]
    
m6.DemandInventoryBalance = pyo.Constraint(m6.IY, m6.J, rule=demand_inventory_balance)

#m6.DemandInventoryBalance.pprint()

# Minimum inventory level constraint at end of month 2
def min_inventory_level(m, j):
    return m.y[2,j] >= m.b

m6.MinInventoryLevel = pyo.Constraint(m6.J, rule=min_inventory_level)

#m6.MinInventoryLevel.pprint()

# Machine line capacity constraints
def machine_line_capacity(m, i, k):
    return sum(m.r[j,k] * m.x[i,j,k] for j in m.J) <= m.l[i,k]

m6.MachineLineCapacity = pyo.Constraint(m6.I, m6.K, rule=machine_line_capacity)

#m6.pprint()

### Solve the model

In [13]:
results6 = opt.solve(m6)

# Print the results
print("Objective value: $", pyo.value(m6.OBJ))
print("Optimal units of product j produced in month i on line k:", \
      {(i,j,k): pyo.value(m6.x[i,j,k]) for i in m6.I for j in m6.J for k in m6.K if pyo.value(m6.x[i,j,k]) > 0})
print("Optimal inventory of product j at end of month i:", \
      {(i,j): pyo.value(m6.y[i,j]) for i in m6.IY for j in m6.J if pyo.value(m6.y[i,j]) > 0})
print("All other combinations of x[i,j,k] and y[i,j] are 0")

Objective value: $ 45997.54901960784
Optimal units of product j produced in month i on line k: {(1, 1, 1): 3666.6666666666674, (1, 1, 2): 3362.7450980392155, (1, 2, 1): 2083.333333333333, (2, 1, 2): 6470.588235294117, (2, 2, 1): 4166.666666666667}
Optimal inventory of product j at end of month i: {(0, 1): 500.0, (0, 2): 750.0, (1, 1): 2529.411764705883, (1, 2): 833.333333333333, (2, 1): 1000.0, (2, 2): 1000.0}
All other combinations of x[i,j,k] and y[i,j] are 0
